# TIL: Debugging SSH Access to GitHub Enterprise

## The Problem
Cloning a repository failed with "Repository not found" even though I could see it in my browser.

## The Root Cause
My SSH key was linked to my **personal** GitHub account, but the repository belonged to my **Enterprise** account.

## How I Diagnosed It

In [ ]:
# Test which account SSH authenticates as
# ssh -T git@github.com
# Output: "Hi personal-username!" ← Wrong account!

# See which key SSH is using (verbose mode)
# ssh -vT git@github.com
# Look for: "Server accepts key: /path/to/keyfile"

## The Fix

1. **Add SSH key to Enterprise account** — Settings → SSH and GPG keys → New SSH key
2. **Authorize for SSO** — Click "Configure SSO" → Authorize for your organization
3. **Update `~/.ssh/config`** to use the correct key
4. **Clear cached keys** and verify

In [ ]:
# SSH config should look like:
# Host github.com
#   HostName github.com
#   User git
#   IdentityFile ~/.ssh/id_ed25519

In [ ]:
# Clear cached keys
# ssh-add -D

# Verify correct account
# ssh -T git@github.com
# Output: "Hi enterprise-username!" ← Correct!

## Key Commands Reference

In [ ]:
# View SSH files
# ls -ltr ~/.ssh/
# cat ~/.ssh/config

# Generate new key
# ssh-keygen -t ed25519 -f ~/.ssh/id_ed25519 -C "email" -N ""

# Show key fingerprint
# ssh-keygen -lf ~/.ssh/id_ed25519.pub

# Copy public key to clipboard (Mac)
# pbcopy < ~/.ssh/id_ed25519.pub

# Test SSH with specific key
# ssh -i ~/.ssh/id_ed25519 -T git@github.com

## Key Insight

GitHub shows "Repository not found" for both non-existent repos AND repos you don't have access to — for security reasons. So when debugging, always verify **which account** your SSH key authenticates as!

In [ ]:
#| hide

import subprocess
from pathlib import Path
from dialoghelper import curr_dialog

def deploy_notebook():
    nb_name = Path(curr_dialog()['name']).name + '.ipynb'
    src = f'/app/data/{curr_dialog()["name"]}.ipynb'
    dst = '/app/data/publish/portfolio/static/'
    print(nb_name)
    
    # Copy notebook to static folder
    subprocess.run(['cp', src, dst])
    
    # Deploy with plash
    subprocess.run(['plash_deploy'], cwd='/app/data/publish/portfolio')

In [ ]:
#| hide
deploy_notebook()